In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms, datasets, models
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Define the path to your dataset
path = '/home/rishabh.mondal/R_P_project/Faulty_solar_panel'

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=path, transform=transform)

# Define the number of splits for cross-validation
n_splits = 4  # 4-fold cross-validation

# Initialize stratified k-fold cross-validation
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [3]:
model = models.resnet18(pretrained=True)
model

/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# Lists to store metrics for plotting
train_losses = []
val_losses = []
accuracies = []

# Training loop for each fold
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(dataset.samples, dataset.targets)):
    print(f"Training Fold {fold + 1}/{n_splits}")

    # Create data loaders for the current fold
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=264, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=264, sampler=val_sampler)

    print(len(train_loader), len(val_loader))

    # Define the Resnet18 model
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(512, len(dataset.classes))

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Move the model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    num_epochs = 50

    for epoch in range(num_epochs):
        model.train()
        total_correct = 0
        total_samples = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = total_correct / total_samples
        train_losses.append(train_loss)

        # Validation
        if epoch % 10 == 9:  # Print results every 10 epochs
            model.eval()
            with torch.no_grad():
                val_loss = 0.0
                correct = 0
                total = 0
                all_predicted = []
                all_labels = []

                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)

                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    all_predicted.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

                val_losses.append(val_loss / len(val_loader))
                accuracy = correct / total
                accuracies.append(accuracy)

                # Calculate precision, recall, and F1 score
                precision = precision_score(all_labels, all_predicted, average='weighted')
                recall = recall_score(all_labels, all_predicted, average='weighted')
                f1 = f1_score(all_labels, all_predicted, average='weighted')

                print(f'Fold: {fold + 1}, Epoch {epoch + 1}/{num_epochs}, '
                      f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
                      f'Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {accuracy:.4f}, '
                      f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    # # Plotting loss and accuracy curves
    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 2, 1)
    # plt.plot(train_losses, label='Train Loss')
    # plt.plot(val_losses, label='Validation Loss')
    # plt.xlabel('Epochs')
    # plt


Training Fold 1/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold: 1, Epoch 10/50, Train Loss: 0.0115, Train Accuracy: 0.9970, Val Loss: 0.7285, Val Accuracy: 0.8423, Precision: 0.8598, Recall: 0.8423, F1 Score: 0.8432
Fold: 1, Epoch 20/50, Train Loss: 0.0083, Train Accuracy: 0.9940, Val Loss: 0.5086, Val Accuracy: 0.8739, Precision: 0.8750, Recall: 0.8739, F1 Score: 0.8730
Fold: 1, Epoch 30/50, Train Loss: 0.0054, Train Accuracy: 0.9970, Val Loss: 0.4662, Val Accuracy: 0.8784, Precision: 0.8795, Recall: 0.8784, F1 Score: 0.8781
Fold: 1, Epoch 40/50, Train Loss: 0.0072, Train Accuracy: 0.9970, Val Loss: 0.4486, Val Accuracy: 0.8784, Precision: 0.8794, Recall: 0.8784, F1 Score: 0.8778
Fold: 1, Epoch 50/50, Train Loss: 0.0057, Train Accuracy: 0.9955, Val Loss: 0.4626, Val Accuracy: 0.8739, Precision: 0.8738, Recall: 0.8739, F1 Score: 0.8733
Training Fold 2/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold: 2, Epoch 10/50, Train Loss: 0.0213, Train Accuracy: 0.9955, Val Loss: 1.5683, Val Accuracy: 0.7285, Precision: 0.7437, Recall: 0.7285, F1 Score: 0.7182
Fold: 2, Epoch 20/50, Train Loss: 0.0013, Train Accuracy: 1.0000, Val Loss: 0.8174, Val Accuracy: 0.8416, Precision: 0.8454, Recall: 0.8416, F1 Score: 0.8419
Fold: 2, Epoch 30/50, Train Loss: 0.0002, Train Accuracy: 1.0000, Val Loss: 0.8640, Val Accuracy: 0.8416, Precision: 0.8464, Recall: 0.8416, F1 Score: 0.8421
Fold: 2, Epoch 40/50, Train Loss: 0.0002, Train Accuracy: 1.0000, Val Loss: 0.8770, Val Accuracy: 0.8326, Precision: 0.8391, Recall: 0.8326, F1 Score: 0.8338
Fold: 2, Epoch 50/50, Train Loss: 0.0001, Train Accuracy: 1.0000, Val Loss: 0.8680, Val Accuracy: 0.8416, Precision: 0.8465, Recall: 0.8416, F1 Score: 0.8425
Training Fold 3/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold: 3, Epoch 10/50, Train Loss: 0.0191, Train Accuracy: 0.9955, Val Loss: 1.0024, Val Accuracy: 0.8235, Precision: 0.8388, Recall: 0.8235, F1 Score: 0.8219
Fold: 3, Epoch 20/50, Train Loss: 0.0095, Train Accuracy: 0.9955, Val Loss: 0.9524, Val Accuracy: 0.8190, Precision: 0.8477, Recall: 0.8190, F1 Score: 0.8201
Fold: 3, Epoch 30/50, Train Loss: 0.0119, Train Accuracy: 0.9970, Val Loss: 2.5193, Val Accuracy: 0.6335, Precision: 0.7952, Recall: 0.6335, F1 Score: 0.6044
Fold: 3, Epoch 40/50, Train Loss: 0.0368, Train Accuracy: 0.9880, Val Loss: 1.5225, Val Accuracy: 0.7376, Precision: 0.7550, Recall: 0.7376, F1 Score: 0.7377
Fold: 3, Epoch 50/50, Train Loss: 0.0072, Train Accuracy: 0.9970, Val Loss: 0.9319, Val Accuracy: 0.7964, Precision: 0.8051, Recall: 0.7964, F1 Score: 0.7976
Training Fold 4/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold: 4, Epoch 10/50, Train Loss: 0.0242, Train Accuracy: 0.9940, Val Loss: 0.8878, Val Accuracy: 0.8326, Precision: 0.8495, Recall: 0.8326, F1 Score: 0.8349
Fold: 4, Epoch 20/50, Train Loss: 0.0121, Train Accuracy: 0.9970, Val Loss: 1.1346, Val Accuracy: 0.8552, Precision: 0.8614, Recall: 0.8552, F1 Score: 0.8537
Fold: 4, Epoch 30/50, Train Loss: 0.0042, Train Accuracy: 0.9985, Val Loss: 0.7384, Val Accuracy: 0.8597, Precision: 0.8664, Recall: 0.8597, F1 Score: 0.8591
Fold: 4, Epoch 40/50, Train Loss: 0.0036, Train Accuracy: 0.9985, Val Loss: 0.7737, Val Accuracy: 0.8733, Precision: 0.8769, Recall: 0.8733, F1 Score: 0.8719
Fold: 4, Epoch 50/50, Train Loss: 0.0036, Train Accuracy: 0.9985, Val Loss: 0.7771, Val Accuracy: 0.8688, Precision: 0.8699, Recall: 0.8688, F1 Score: 0.8670


In [5]:
# Lists to store metrics for plotting
train_losses = []
val_losses = []
accuracies = []

# Training loop for each fold
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(dataset.samples, dataset.targets)):
    print(f"Training Fold {fold + 1}/{n_splits}")

    # Create data loaders for the current fold
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=264, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=264, sampler=val_sampler)

    print(len(train_loader), len(val_loader))

    # Define the ResNet18 model
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(512, len(dataset.classes))

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Move the model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    num_epochs = 50

    for epoch in range(num_epochs):
        model.train()
        total_correct = 0
        total_samples = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = total_correct / total_samples
        train_losses.append(train_loss)

        # Validation
        if epoch % 10 == 9:  # Print results every 10 epochs
            model.eval()
            with torch.no_grad():
                val_loss = 0.0
                correct = 0
                total = 0
                all_predicted = []
                all_labels = []

                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)

                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    all_predicted.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

                val_losses.append(val_loss / len(val_loader))
                accuracy = correct / total
                accuracies.append(accuracy)

                # Calculate precision, recall, and F1 score
                precision = precision_score(all_labels, all_predicted, average='weighted')
                recall = recall_score(all_labels, all_predicted, average='weighted')
                f1 = f1_score(all_labels, all_predicted, average='weighted')

                print(f'Fold: {fold + 1}, Epoch {epoch + 1}/{num_epochs}, '
                      f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
                      f'Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {accuracy:.4f}, '
                      f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    # # Plotting loss and accuracy curves
    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 2, 1)
    # plt.plot(train_losses, label='Train Loss')
    # plt.plot(val_losses, label='Validation Loss')
    # plt.xlabel('Epochs')
    # plt


Training Fold 1/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 1, Epoch 10/50, Train Loss: 0.4525, Train Accuracy: 0.8522, Val Loss: 3.9141, Val Accuracy: 0.3378, Precision: 0.1869, Recall: 0.3378, F1 Score: 0.2216
Fold: 1, Epoch 20/50, Train Loss: 0.1545, Train Accuracy: 0.9668, Val Loss: 2.8126, Val Accuracy: 0.4099, Precision: 0.5078, Recall: 0.4099, F1 Score: 0.3960
Fold: 1, Epoch 30/50, Train Loss: 0.0811, Train Accuracy: 0.9774, Val Loss: 1.6240, Val Accuracy: 0.6532, Precision: 0.6801, Recall: 0.6532, F1 Score: 0.6577
Fold: 1, Epoch 40/50, Train Loss: 0.0106, Train Accuracy: 0.9925, Val Loss: 1.7525, Val Accuracy: 0.6126, Precision: 0.6634, Recall: 0.6126, F1 Score: 0.6085
Fold: 1, Epoch 50/50, Train Loss: 0.0090, Train Accuracy: 0.9970, Val Loss: 1.2442, Val Accuracy: 0.7568, Precision: 0.7688, Recall: 0.7568, F1 Score: 0.7573
Training Fold 2/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 2, Epoch 10/50, Train Loss: 0.4340, Train Accuracy: 0.8675, Val Loss: 4.7682, Val Accuracy: 0.2986, Precision: 0.2251, Recall: 0.2986, F1 Score: 0.1830
Fold: 2, Epoch 20/50, Train Loss: 0.1191, Train Accuracy: 0.9639, Val Loss: 1.7556, Val Accuracy: 0.6063, Precision: 0.6074, Recall: 0.6063, F1 Score: 0.5852
Fold: 2, Epoch 30/50, Train Loss: 0.0174, Train Accuracy: 0.9970, Val Loss: 1.6725, Val Accuracy: 0.7059, Precision: 0.7071, Recall: 0.7059, F1 Score: 0.7007
Fold: 2, Epoch 40/50, Train Loss: 0.0041, Train Accuracy: 0.9985, Val Loss: 1.6212, Val Accuracy: 0.6923, Precision: 0.6961, Recall: 0.6923, F1 Score: 0.6911
Fold: 2, Epoch 50/50, Train Loss: 0.0010, Train Accuracy: 1.0000, Val Loss: 1.6382, Val Accuracy: 0.7466, Precision: 0.7408, Recall: 0.7466, F1 Score: 0.7389
Training Fold 3/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 3, Epoch 10/50, Train Loss: 0.4622, Train Accuracy: 0.8494, Val Loss: 3.7550, Val Accuracy: 0.2851, Precision: 0.2082, Recall: 0.2851, F1 Score: 0.1685
Fold: 3, Epoch 20/50, Train Loss: 0.0796, Train Accuracy: 0.9759, Val Loss: 1.3271, Val Accuracy: 0.6471, Precision: 0.7364, Recall: 0.6471, F1 Score: 0.6501
Fold: 3, Epoch 30/50, Train Loss: 0.0233, Train Accuracy: 0.9970, Val Loss: 2.3054, Val Accuracy: 0.5520, Precision: 0.7243, Recall: 0.5520, F1 Score: 0.5059
Fold: 3, Epoch 40/50, Train Loss: 0.1068, Train Accuracy: 0.9669, Val Loss: 2.1196, Val Accuracy: 0.5520, Precision: 0.6316, Recall: 0.5520, F1 Score: 0.5238
Fold: 3, Epoch 50/50, Train Loss: 0.0278, Train Accuracy: 0.9955, Val Loss: 1.5041, Val Accuracy: 0.6697, Precision: 0.7259, Recall: 0.6697, F1 Score: 0.6683
Training Fold 4/4
3 1


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 4, Epoch 10/50, Train Loss: 0.3791, Train Accuracy: 0.8675, Val Loss: 5.5074, Val Accuracy: 0.2489, Precision: 0.3912, Recall: 0.2489, F1 Score: 0.1375
Fold: 4, Epoch 20/50, Train Loss: 0.0488, Train Accuracy: 0.9864, Val Loss: 1.9650, Val Accuracy: 0.5068, Precision: 0.5793, Recall: 0.5068, F1 Score: 0.4528
Fold: 4, Epoch 30/50, Train Loss: 0.0119, Train Accuracy: 0.9985, Val Loss: 1.4084, Val Accuracy: 0.7059, Precision: 0.7718, Recall: 0.7059, F1 Score: 0.7044
Fold: 4, Epoch 40/50, Train Loss: 0.0078, Train Accuracy: 0.9985, Val Loss: 1.0324, Val Accuracy: 0.7873, Precision: 0.7915, Recall: 0.7873, F1 Score: 0.7864
Fold: 4, Epoch 50/50, Train Loss: 0.0084, Train Accuracy: 0.9940, Val Loss: 1.1635, Val Accuracy: 0.7692, Precision: 0.7785, Recall: 0.7692, F1 Score: 0.7689
